<a href="https://www.kaggle.com/code/mohitsingh351/brain-tumor-mri-classification?scriptVersionId=190235820" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Import necessary libraries**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

2024-07-29 04:35:45.678736: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 04:35:45.678871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 04:35:45.832192: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **Collecting File Paths and Labels from Directory**

In [2]:
parent_dir = "/kaggle/input/brain-mri-images-for-brain-tumor-detection"

filepaths = []
labels = []

# Directory containing the "brain_tumor_dataset" folder
directory = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset"
folders = os.listdir(directory)  #['no', 'yes']

for folder in folders:
    file_path = os.path.join(parent_dir, folder)
    images = os.listdir(file_path)
    for image in images:
        image_path = os.path.join(file_path, image)
        filepaths.append(image_path)
        labels.append(folder)

# **Creating DataFrame from File Paths and Labels**

In [3]:
file_path_series = pd.Series(filepaths, name="filepath")
label_series = pd.Series(labels, name="label")

df_train = pd.concat([file_path_series, label_series], axis=1)

In [4]:
print(f"Shape of 'df_train': {df_train.shape}")
df_train

Shape of 'df_train': (253, 2)


,filepath,label
0,/kaggle/input/brain-mri-images-for-brain-tumor...,no
1,/kaggle/input/brain-mri-images-for-brain-tumor...,no
2,/kaggle/input/brain-mri-images-for-brain-tumor...,no
3,/kaggle/input/brain-mri-images-for-brain-tumor...,no
4,/kaggle/input/brain-mri-images-for-brain-tumor...,no
...,...,...
248,/kaggle/input/brain-mri-images-for-brain-tumor...,yes
249,/kaggle/input/brain-mri-images-for-brain-tumor...,yes
250,/kaggle/input/brain-mri-images-for-brain-tumor...,yes
251,/kaggle/input/brain-mri-images-for-brain-tumor...,yes


# **Splitting Data into Training, Validation, and Test Sets**

In [5]:
# Spliting in Train and Dummy 
train_df , dummy_df = train_test_split(df_train ,train_size = 0.8 , shuffle = True ,random_state = 42 )
# Spliting in Validation and Test From Dummy 
valid_df , test_df = train_test_split(dummy_df ,test_size= 0.5 , shuffle = True ,random_state = 42)

In [6]:
print(f"The shape of train data is: {train_df.shape}")
print(f"The shape of validation data is: {valid_df.shape}")
print(f"The shape of test data is: {test_df.shape}")

The shape of train data is: (202, 2)
The shape of validation data is: (25, 2)
The shape of test data is: (26, 2)


# **Data Augmentation and Generators for Training, Validation, and Test Sets**

In [7]:
image_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=20, zoom_range=0.2, horizontal_flip=True, shear_range=0.15
)

validation_test_datagenerator = ImageDataGenerator(rescale=1./255)

print("Training data")
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='filepath',
    y_col='label',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='binary'
)

print('Validation data')
validation_generator = validation_test_datagenerator.flow_from_dataframe(
    valid_df,
    x_col='filepath',
    y_col='label',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='binary'
)

print('Test data')
test_generator = validation_test_datagenerator.flow_from_dataframe(
    test_df,
    x_col='filepath',
    y_col='label',
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Training data
Found 202 validated image filenames belonging to 2 classes.
Validation data
Found 25 validated image filenames belonging to 2 classes.
Test data
Found 26 validated image filenames belonging to 2 classes.


# **Building and Compiling the VGG16-Based Model**

In [8]:
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(image_size, image_size, 3))

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

base_model.trainable = False  # Freeze the base model

model.compile(optimizer="Adam", loss='binary_crossentropy', metrics=['accuracy'])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# **Training the Model with Early Stopping**

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    callbacks=[early_stopping]
)

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 62s 8s/step - accuracy: 0.5883 - loss: 1.5319 - val_accuracy: 0.5200 - val_loss: 0.7939
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 80s 8s/step - accuracy: 0.7428 - loss: 0.8639 - val_accuracy: 0.5600 - val_loss: 0.9406
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 59s 8s/step - accuracy: 0.7146 - loss: 1.0398 - val_accuracy: 0.6800 - val_loss: 0.5276
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 58s 9s/step - accuracy: 0.7194 - loss: 0.6220 - val_accuracy: 0.8400 - val_loss: 0.4833
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 58s 9s/step - accuracy: 0.7013 - loss: 0.6494 - val_accuracy: 0.8000 - val_loss: 0.5071
Epoch 6/20
6/7 ━━━━━━━━━━━━━━━━━━━━ 6s 7s/step - accuracy: 0.7923 - loss: 0.4733 

# **Evaluating Model Performance on Test Set**

In [ ]:
loss, accuracy = model.evaluate(test_generator)

In [ ]:
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# **Generating and Processing Predictions on Test Set**

In [ ]:
predictions = model.predict(test_generator)

In [ ]:
predicted_labels = (predictions > 0.5).astype(int)

# **Confusion Matrix**

In [ ]:
confusion_matrix(test_generator.labels, predicted_labels)

# **Classification Report**

In [ ]:
print(classification_report(test_generator.labels, predicted_labels, target_names=['No Tumor', 'Tumor']))